In [2]:
# 구글 제미나이 설치
# %pip install google-generativeai

In [2]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()

# gemini api key 사용
genai.configure(api_key=os.environ['GEMINI_OPEN_API_KEY'])


In [7]:
# 모델 생성하기
generation_config = {
    'temperature':1,
    'top_p':0.95,
    'top_k':64,
    'max_output_tokens':8192,
    'response_mime_type':'text/plain',
}

model = genai.GenerativeModel(
    model_name='gemini-1.5-flash',
    generation_config=generation_config,
)

In [8]:
# gemini 사용해보기
chat_session = model.start_chat(
    # 미리 히스토리 설정해서 채팅흔적 남기기
    history=[
        {
            'role':'user',
            'parts':[
                '다음 문장의 키워드를 추출해주세요.',
                '"제주시 한림읍에 있는 카페 중 30대 이용 비중이 가장 높은 곳은?"\n',
            ],
        },
        {
            'role':'model',
            'parts':[
                '## 키워드 추출\n\n- **제주시 한림읍** : 지역정보\n- **카페** : 업종\n- **30대** : 고객 연령대\n- **이용 비중** : 분석기준 (매출, 인기)'
            ],
        },
    ]
)

response = chat_session.send_message('제주시 노형동에 있는 단품요리 전문점 중 이용건수가 상위 10%에 속하고 현지인 이용 비중이 가장 높은 곳은?')
print(response.text)

## 키워드 추출

* **제주시 노형동**: 지역 정보
* **단품요리 전문점**: 업종
* **이용 건수 상위 10%**: 인기 기준
* **현지인 이용 비중 가장 높은**: 고객층 기준 



In [10]:
# LLM을 이용한 키워드 추출 함수
def extract_keywords_from_text(query):
    # Gemini LLM 프롬프트 생성
    prompt_template = f'''
    문장에서 중요한 정보를 JSON 형식으로 추출해주세요. 키는 아래의 리스트 중 문맥상 가장 적합한 항목을 사용해주세요.
    
    keys = [
        '가맹점명', '개설일자', '업종', '주소', '이용건수구간', '이용금액구간', '건당평균이용금액구간',
        '월_이용건수비중', '화_이용건수비중', '수_이용건수비중', '목_이용건수비중', '금_이용건수비중', '토_이용건수비중',
        '일_이용건수비중', '05-11시_이용건수비중', '12-13시_이용건수비중', '14-17시_이용건수비중',
        '18-22시_이용건수비중', '23-04시_이용건수비중', '현지인_이용건수비중', '남성_이용건수비중',
        '여성_이용건수비중', '10-20대_회원비중', '30대_회원비중', '40대_회원비중', '50대_회원비중',
        '60대이상_회원비중'
    ]
    
    <규칙>
    1. "비중"으로 끝나는 항목의 값이 `높은` 것을 언급하면 상위 3개를 나타내는 "상위 3개" 형식으로, `낮은` 것을 언급하면 하위 3개를 나타내는 "하위 3개" 형식으로 표시해주세요.
    2. "구간"으로 끝나는 항목 (예시: "이용건수구간", "이용금액구간", "건당평균이용금액구간")은 다음과 같은 값으로 변환해주세요:
        - "상위 10%" : "상위 10% 이하",
        - "10-25%" : "10-25%",
        - "25-50%" : "25-50%",
        - "50-75%" : "50-75%",
        - "75-90%" : "75-90%",
        - "90% 미만" : "90% 초과"
    3. "주소"가 언급되면 key는 "주소", value는 해당 주소로 지정해주세요.
    4. "업종"이 입력되면,
        ['가정식', '커피', '분식', '단품요리 전문', '치킨', '중식', '맥주/요리주점', '양식', '베이커리',
        '아이스크림/빙수', '일식', '샌드위치/토스트', '구내식당/푸드코트', '피자', '떡/한과', '민속주점',
        '햄버거', '동남아/인도음식', '꼬치구이', '패밀리 레스토랑', '차', '도시락', '야식', '부페',
        '도너츠', '스테이크', '기타세계요리', '기사식당', '주스', '포장마차']
        위의 리스트의 요소 중 하나로 변환해주세요. 없다면 유사한 것으로 변환해주세요.
        (예시: 업종이 "단품요리 전문점"으로 입력되면, "단품요리 전문"으로 변환해주세요.)
        
    <예시>
    사용자 입력 : "제주시 노형동에 있는 단품요리 전문점 중 이용건수가 상위 10%에 속하고 현지인 이용 비중이 가장 높은 곳은?"
    출력 :
    {{
        "주소":"제주시 노형동",
        "업종":"단품요리 전문",
        "이용건수구간":"상위 10% 이하",
        "현지인_이용건수비중":"상위 3개",
    }}
    
    사용자 입력 : "{query}"
    '''
    
    # 대화 세션 시작
    chat_session = model.start_chat(
        history=[
            {
                'role':'user',
                'parts':[prompt_template],
            }
        ]
    )
    
    # LLM에 프롬프트 전달 및 키워드 추출
    response = chat_session.send_message(query)
    
    # 키워드를 쉼표로 구분한 결과를 리스트로 변환하기
    keywords = response.text.strip().replace("```json", "").replace("```", "")
    
    return keywords

# 테스트 쿼리
query = "애월읍에 있는 주점 중 20시 이용건수비중이 낮고, 현지인 비중이 높은, 이용금액구간이 상위 10% 정도인 곳은?"

# 키워드 추출
keywords = extract_keywords_from_text(query)
print(keywords)


{
  "주소": "애월읍",
  "업종": "맥주/요리주점",
  "18-22시_이용건수비중": "하위 3개",
  "현지인_이용건수비중": "상위 3개",
  "이용금액구간": "상위 10% 이하"
}



In [11]:
import json

# 연습
query = '제주시 노형동에 있는 단품요리 전문점 중 이용건수가 상위 10%에 속하고 현지인 이용 비중이 가장 높은 곳은?'

# 키워드 추출
keywords = extract_keywords_from_text(query)
print(keywords)

try:
    keywords_json = json.loads(keywords)
    
except json.JSONDecodeError as e:
    print(f'JSON 변환 오류: {e}')
    keywords_json = None
    
keywords_json


{
  "주소": "제주시 노형동",
  "업종": "단품요리 전문",
  "이용건수구간": "상위 10% 이하",
  "현지인_이용건수비중": "상위 3개"
}



{'주소': '제주시 노형동',
 '업종': '단품요리 전문',
 '이용건수구간': '상위 10% 이하',
 '현지인_이용건수비중': '상위 3개'}

In [ ]:
# 주소로 우선 필터링하기
